In [1]:
import csv
import numpy as np
import nltk
import pickle
from nltk import word_tokenize

In [2]:
csvfile = open('question_train.csv', 'r')
reader = csv.reader(csvfile, delimiter=',', quotechar='"', lineterminator='<CR>')

In [3]:
csvfile.seek(0);
#cols = list(zip(np.arange(21),next(reader)))
j = 0
for i, q in zip(np.arange(21), next(reader)):
    if j == 3: l = "\n"; j = 0;
    else: l = ""; j += 1
        
    print('{0:2}: {1:25}'.format(i,q), end=l)

 0: question_id               1: user_id                   2: sms_guru_id               3: category_main_id         
 4: question                  5: description               6: tags                      7: categories               
 8: url                       9: rating_count_positive    10: rating_count_negative    11: answer_count             
12: reported                 13: answered                 14: active                   15: deleted                  
16: seo_locked               17: editor_locked            18: editor_id                19: created_at               
20: updated_at               

In [11]:
catfile = open('category.csv', 'r')
cat_reader = csv.reader(catfile, delimiter=',', quotechar='"', lineterminator='\n')

In [30]:
catfile.seek(0);


## Showing what colums mean ###
j = 0
for i, q in zip(np.arange(21), next(cat_reader)):
    if j == 0: l = "\n"; j = 0;
    else: l = ""; j += 1
        
    print('{0:2}: {1:25}'.format(i,q), end=l)
### .......................###

print("\n\n")

category_dict = {} # Dictionary which has as key: category_id, ...
#..- and as values: (category_name, dictionary of subcategories)
subcategory_dict = {}

for row in cat_reader:
    if int(row[1]) == 0:
        category_dict[int(row[0])] = [row[2], {}]

catfile.seek(0); next(cat_reader);

for row in cat_reader:
    if int(row[1]) != 0:
        category_dict[int(row[1])][1][int(row[0])] = row[2]

for cat_id, cat in category_dict.items():
    print(cat_id, cat[0])
    for subcat in cat[1].items():
        print("\t", subcat[0], subcat[1])
    print("")
        

 0: category_id              
 1: parent_id                
 2: category_name            
 3: headline                 
 4: title                    
 5: description              
 6: meta_description         
 7: meta_tag                 
 8: url                      
 9: level                    
10: sort_id                  
11: use_count                
12: active                   
13: created_at               
14: updated_at               



7 Film & Musik
	 20 Musik-Bands
	 21 Musik-Charts
	 22 TV-Serien
	 23 Film & Fernsehen

8 Stars & Promis
	 72 Popstars
	 73 Filmstars/Schauspieler
	 70 Prominente
	 71 Politiker

9 Computer & PC
	 68 PC-Programme
	 69 Internet

10 Alltag
	 24 Regionales
	 25 Wetter
	 84 Tipps & Tricks

11 Namensforschung
	 26 Namensbedeutung
	 27 Namensherkunft

12 Literatur & Sprache
	 28 Übersetzung
	 29 Wortbedeutung
	 30 Bücher
	 85 Mythen & Gedichte

13 Schule
	 32 Biologie
	 33 Geschichte
	 34 Chemie
	 35 Physik
	 36 Mathematik
	 86 Deutsch
	 31 Geogra

# checking for ill-shaped lines

In [7]:
csvfile.seek(0);
i = 0 #count total row number
j = 0 #count false row number

for row in reader:
    i += 1
    if(len(row) != 21):
        #print('line size error in line {} \n'.format(reader.line_num))
        #print('line size is: {} \n'.format(len(row)))
        #print(*row)
        #print('\n')
        j += 1

print("""Total number of rows: {0}
Number of rows with wrong length: {1}:
    fraction: {2:.2f} %""".format(i,j,(j/i)*100))

Total number of rows: 16070
Number of rows with wrong length: 57:
    fraction: 0.35 %


# Reading out questions and tokenizing, checking vocabulary

In [15]:
csvfile.seek(0);
next(reader)

questions = []
vocabulary = set([])

# Set this parameter to TRUE if you want to read through
# all questions again, elsewise from file set to FALSE.
NewRead = False 

if NewRead:
    for row in reader:
        if(len(row) == 21):
            words = word_tokenize(row[4].lower())
            questions.append( (row[0], words) )
            vocabulary = vocabulary.union( words )
        
    ## Saving into pickle files
    q_file, v_file = open('questions.pkl', 'wb'), open('vocabulary.pkl', 'wb')
    pickle.dump(questions, q_file)
    pickle.dump(vocabulary, v_file)
    
else:
    ## Loading from pickle files
    q_file, v_file = open('questions.pkl', 'rb'), open('vocabulary.pkl', 'rb')
    questions, vocabulary = pickle.load(q_file), pickle.load(v_file)

# Checking Vocabulary

In [17]:
totNum = sum([len(q[1]) for q in questions])
vocNum = len(vocabulary)
print("""The total number of words is {0}. 
The volume of the vocabulary is {1}.
That makes an percentiage of {2:.2}""".format(totNum,vocNum,vocNum/totNum))

The total number of words is 140282. 
The volume of the vocabulary is 19682.
That makes an percentiage of 0.14


In [75]:
allwordlist = [w for q in questions for w in q[1]]
fd = nltk.FreqDist(allwordlist)

In [102]:
n = 50
delta = 0
rows = 5
epc = int(n/rows)
a = np.arange(n) + delta

for j in range(epc):
    row = [fd.most_common(n)[i] for i in a[j::epc]]
    for v in row:
        print(("{0:>10} x {1:<6}").format(*v),end="")
    print("")

         ? x 11576   bedeutet x 1465       kommt x 947           '' x 649         oder x 506   
       der x 4693         das x 1386          wo x 879       welche x 642         mein x 481   
       wie x 3999         hat x 1351         wer x 810         gibt x 593          bei x 478   
       was x 3919         von x 1260        eine x 781          den x 587         viel x 455   
       ist x 3470         ein x 1135         mit x 769           er x 574         wenn x 452   
       die x 2366          es x 1112        kann x 749          für x 562         welt x 430   
       und x 2255         man x 1075       woher x 728        heißt x 554          sie x 423   
       ich x 2011           , x 1061         auf x 701         sind x 548        haben x 415   
        in x 1717           . x 1004       warum x 687           im x 542        viele x 415   
      name x 1652        wann x 997           am x 683           zu x 530          aus x 380   


In [104]:
vocabArr = sorted(list(vocab))

n = 50
delta = 2000
rows = 5
epc = int(n/rows)
a = np.arange(n) + delta

for j in range(epc):
    row = [vocabArr[i] for i in a[j::epc]]
    print(("- {:20}"*len(row)).format(*row))

- anschlag            - anstecken           - antikatalytische    - antonin             - antwortetst         
- anschliesend        - ansteckend          - antike              - antonio             - anubis              
- anschließen         - anstellen           - antimaterie         - antony              - anunis              
- anschlägt           - antarktis           - antipinoxe          - antreten            - anus                
- anschrift           - anteil              - antisemetismus      - antrieb             - anwalt              
- anschwillt          - antenne             - antisemitismus      - antrittsvorlesung   - anwendungen         
- ansehen             - antennenverhältnis  - antje               - anträge             - anwendungsgebiete   
- ansprechen          - anthony             - anton               - antwort             - anwesen             
- anspruch            - antibabypille       - antonella           - antworten           - anwesend            
-

# Playing area

In [125]:
import re
for q in questions[:10]:
    print([w for w in q[1] if re.search('^W',w)])

['Was']
['WIEVIEL']
['Was']
[]
[]
['Wie']
['Wie']
['Wie']
[]
['Was']


In [21]:
for x in range(1,11):
    print(repr(x).zfill(5), repr(x**2).ljust(5),repr(x**3).ljust(5))

00001 1     1    
00002 4     8    
00003 9     27   
00004 16    64   
00005 25    125  
00006 36    216  
00007 49    343  
00008 64    512  
00009 81    729  
00010 100   1000 


In [13]:
a = np.arange(5)
b = np.arange(5) +5

In [18]:
list(zip(a,b))

[(0, 5), (1, 6), (2, 7), (3, 8), (4, 9)]

In [108]:
print('Hällülöliös\
    ')

Hällülöliös    


In [109]:
nacute = '\u0144'

In [110]:
nacute

'ń'

In [33]:
from nltk.corpus import names

In [42]:
type(names.words('male.txt'))

list

In [85]:
"Hallo".lower()

'hallo'

# Helps

In [128]:
a = np.arange(100)
[b[a[::3]-i] for i in range(3)]

[array([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48,
        51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99]),
 array([99,  2,  5,  8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47,
        50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98]),
 array([98,  1,  4,  7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46,
        49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97])]

In [141]:
a = ['car', 'house', 'dor']
b = ['Cat', 'Dog', 'Mouse']
list(zip(*[a,b]))

[('car', 'Cat'), ('house', 'Dog'), ('dor', 'Mouse')]